# Exploring geometries with `Folium`

Today, we will be exploring visualizing geographical data (latitude and longitude specifically) with Folium,
on the example of a bike-sharing service data from Cyclistic.

# Import libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os, glob
import requests

import time
import math

import pandas as pd
import numpy as np
import sklearn

import matplotlib.pyplot as plt
import seaborn as sns

import folium
from folium import plugins

# Import data

The data can be found via this [link](https://divvy-tripdata.s3.amazonaws.com/index.html).

In [ ]:
df = pd.read_csv('data/202406-divvy-tripdata.csv')

In [ ]:
df.head()

In [ ]:
df.describe()

The most recent file, dated June 2024, has 0.7 M observations. Let's import the rest of the data for the past year:

In [ ]:
path = r'data' # use your path
all_files = glob.glob(path + "/*.csv")

In [ ]:
for file in sorted(all_files):
    file_size = os.path.getsize(file)
    print(file, file_size)

Just from the file sizes, we can observe that there was a **decline in bike shares during winter** season (since more data, or file size, would mean more rentals).

In [ ]:
# create list to append to
li = []

# loop through file names in the variable named 'all_files'
for filename in all_files:
    df = pd.read_csv(filename) # skiprows=1, index_col=None
    li.append(df)

In [ ]:
data = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
data.sample(5)

In [ ]:
data.info()

Now we have over 5.7 M observations.

---

# Preprocessing

### `ride_id`: removing duplicates

In [ ]:
len(data['ride_id'].unique()), data['ride_id'].isna().sum()

Seems like all rides' IDs are **not** unique, although there are no missing values.

Let's check if there are duplicates or several bikes linked to the same ID:

In [ ]:
data['ride_id'].value_counts()

In [ ]:
n_all, n_unique = len(data), len(data['ride_id'].value_counts())

print(f'There are {n_all - n_unique} possible duplicates.')

In [ ]:
data[data['ride_id'] == 'FEA150E7A56F187E']

In [ ]:
data[data['ride_id'] == 'F2E698ECB05C43D7']

Seems like all of these are *duplicates*.

In [ ]:
data[['ride_id', 'started_at', 'start_station_id']].groupby(['started_at', 'start_station_id']).count()

Could it be that those with the exact start time AND station are all duplicates? Due to ride registration error, for example.

In [ ]:
data[['ride_id', 'started_at', 'start_station_id']].groupby(['started_at', 'start_station_id']).count().value_counts()

Numbers do not match. We could be adding more features to add to the primary key, BUT notice that `started_at` and `ended_at` **differ only in milliseconds**!

In [ ]:
id_counts = data['ride_id'].value_counts()
duplicated_ids = np.array(id_counts[id_counts.values == 2].index)

def checker(id):
    if id in duplicated_ids:
        return True
    return False

duplicates = data[data['ride_id'].apply(checker)]
duplicates

In [ ]:
cache = {}
cleaned_dups = duplicates.drop_duplicates()
dupped_ind = cleaned_dups['ride_id'].value_counts().index

def checker_specific(ind):
    if ind in dupped_ind:
        if ind in cache:
            return False
        else:
            cache[ind] = True
    return True

In [ ]:
data = data[data['ride_id'].apply(checker_specific)]

Yay! We got rid of the most obvious duplicates.

---

### `started_at` and `ended_at`: handling missing values

Convert columns containing dates to uniform `datetime` type:

In [ ]:
data['started_at'] = pd.to_datetime(data['started_at'], errors='coerce')
data['ended_at'] = pd.to_datetime(data['ended_at'], errors='coerce')

In [ ]:
data.sample(3)

In [ ]:
missing_data = data[(data['started_at'].isna() == True) | (data['ended_at'].isna() == True)]
print(f'There are {len(missing_data)} rows with missing dates, which is {np.round(len(missing_data)/len(data), 2)} from the all data.')
missing_data.sample(3)

It turns out that 710,721 rides have no time data at all! It is slightly over $\frac{1}{10}$ of all our data, so we want to keep these observations. However, since wwe are working with visualizing coordinates, we will be sumplifying this by just removing this data.

In [ ]:
df = data[(data['started_at'].isna() == False) & (data['ended_at'].isna() == False)]
df.sample(3)

---

### `duration`

Let's create new feature `duration` for our new dataset `time_data`:

In [ ]:
df['duration'] = df['ended_at'] - df['started_at']

In [ ]:
df['duration_days'] = df['duration'].dt.days

In [ ]:
np.min(df['duration']), np.mean(df['duration']), np.max(df['duration'])

We see some unreal data, such as -12 days of a single rental.

In [ ]:
sns.violinplot(
    x=df['member_casual'],
    y=df['duration'].dt.days
);

#### 1. Dealing with outliers of  `casual` riders

Very suspicious statistics. Could it be that bikes that were *stolen* were as well not returned to the stations for long period of times? Or casual riders just *could not find* a proper station (since they are not using the service often)? Or maybe, the simply *forgot* to return the bike.

In [ ]:
(df[df['member_casual'] == 'casual'][df['duration'] > pd.to_timedelta('1 days')]
    .sort_values('duration', ascending=False).head())

We can see that at least for five longest rentals bikes were **never returned**, since there are not end station specified!

In [ ]:
sns.violinplot(
    data=df[df['member_casual'] == 'casual'][df['duration'] > pd.to_timedelta('1 days')],
    x='duration_days'
);

Let's remove all rentals when bikes were not returned:

In [ ]:
df = (df[df['end_lat'].isna() == False]
     .sort_values('duration', ascending=False))
df.head(3)

#### 2. Dealing with outliers of annual `member`s

In [ ]:
np.max(df[df['member_casual'] == 'member']['duration'])

In contrast, although members have never exceded a day and a half over the time period, there are *negative* rental times, clearly impossible unless it is a bug.

In [ ]:
sns.violinplot(
    data=df[df['member_casual'] == 'member'][df['duration'] < pd.to_timedelta('0 days')],
    x='duration_days'
);

In [ ]:
len(df[df['member_casual'] == 'member'][df['duration'] < pd.to_timedelta('0 days')])

Let's just remove those observations with negative or zero ride durations:

In [ ]:
df = df[df['duration'] > pd.to_timedelta('0 days')]

#### 3. Adding `duration_mins` subfeature

In [ ]:
df['duration_mins'] = df['duration'].dt.seconds / 60

Let's check if **short rentals** were characteristic for casual riders (who may not know the UI of the app well yet). Assume "short" ride is the rental lasting less than 5 minutes.

In [ ]:
sns.histplot(
    data=df[df['duration_mins'] <= 5],
    x='duration_mins',
    hue='member_casual'
);

Looks like there were more annual members using bikes for short time overall.

### `day_of_week`

In [ ]:
df['day_of_week'] = df['started_at'].dt.dayofweek

### `distance`

In [ ]:
def haversine(lat1, lon1, lat2, lon2):
    """
    Determine the distance (in kilometers) between two points on a sphere
    given their longitudes and latitudes.
    """
    dLat = (lat2 - lat1) * math.pi / 180.0
    dLon = (lon2 - lon1) * math.pi / 180.0
 
    # Convert to radians
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0
    
    a = (pow(math.sin(dLat / 2), 2) +
         pow(math.sin(dLon / 2), 2) *
             math.cos(lat1) * math.cos(lat2));
    rad = 6371
    c = 2 * math.asin(math.sqrt(a))
    
    return rad * c

In [ ]:
df['distance'] = df.apply(
    lambda r: haversine(
        r.start_lat, r.start_lng, r.end_lat, r.end_lng),
    axis=1
)

In [ ]:
sns.histplot(
    data=df.sample(10_000),
    x='distance', # in km
    hue='member_casual'
);

### `avg_speed`

We can also check the **average speed** of riders! Who knows, maybe faster riders are those constantly renting the bikes, and we can convert them into annual members more?

In [ ]:
# Take speed measured in km per hour
df['avg_speed'] = df['distance'] / (df['duration_mins'] / 60)

In [ ]:
df.sample(3)

In [ ]:
df['avg_speed'].min(), df['avg_speed'].mean(), df['avg_speed'].max()

We definitely got some outliers here.

In [ ]:
sns.violinplot(
    data=df,
    x='avg_speed'
);

Let's restrict `avg_speed` by +1 std from the mean:

In [ ]:
df = df[df['avg_speed'] <= df['avg_speed'].mean() + np.std(df['avg_speed'])]
df.sort_values('avg_speed', ascending=False).head(3)

In [ ]:
sns.histplot(
    data=df.sample(1_000_000),
    x='avg_speed', # in km per hour
    hue='member_casual'
);

Seems like annual member are slightly faster on average (centered around 11 km/h) compared to casual riders (centered around 9 km/h).

In [ ]:
np.mean(df[df['member_casual'] == 'casual']['avg_speed']), np.mean(df[df['member_casual'] == 'member']['avg_speed'])

---

Our final dataset:

In [ ]:
df.sample(3)

In [ ]:
len(df)

---

# EDA

## Time Series

### Seasonal trends

Let's dive deeper onto trends over one-year period.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

In [ ]:
data_seasonal = data.copy()
data_seasonal.sample(1)

In [ ]:
data_seasonal['started_at'] = pd.to_datetime(data_seasonal['started_at'], format='mixed')

In [ ]:
data_seasonal = data_seasonal.sort_values('started_at', ascending=True)

Let's create a **unique day identifier** (note that rentals were made from the second half of 2023 to the first half of 2024):

In [ ]:
date_ids = []

for d, m, y in zip(
        data_seasonal['started_at'].dt.day,
        data_seasonal['started_at'].dt.month,
        data_seasonal['started_at'].dt.year):
    identifier = str(d) + ' ' + str(m) + ' ' + str(y)
    date_ids.append(identifier)

data_seasonal['date_id'] = date_ids

In [ ]:
data_seasonal.sample(3)

In [ ]:
date_counts = data_seasonal[['date_id', 'member_casual']].groupby('date_id').count()

def checker_count_by_date(date_id):
    return date_counts.loc[date_id].values[0]

In [ ]:
data_seasonal['date_count'] = data_seasonal['date_id'].apply(checker_count_by_date)

In [ ]:
data_seasonal.head(3)

In [ ]:
plt.figure(figsize=(12, 4))

sns.lineplot(
    data=data_seasonal,
    x='date_id',
    y='date_count',
    hue='member_casual' # doing it wrong; shpuld have calculated counts by groups
)

plt.xticks(rotation='vertical')
plt.show();

## Geometries

### Stations' usage

In [ ]:
len(df['start_station_id'].unique())

In [ ]:
len(df['end_station_id'].unique())

In the data overview, it was stated that there are over six thousand stations in total.

However, around $\frac{1}{4}$ of them were really used for starting and ending the trip.

In [ ]:
np.sum(df['start_station_id'] == df['end_station_id'])

Out of 5.7 M data, 262,567 trips were started and ended on the same spot.

How long did they last and why they could be so? Maybe, it was a new rider who did not know other station? Or it was near a specific location (e.g. a park), where people tend to ride bikes?

In [ ]:
import folium

map = folium.Map(
    location=[62.035454, 129.675476], # Yakutsk (Якутск)
    zoom_start=11
)
map

Let's create a map with initial zoom at the *mean* starting point. 5 million rows are going to take a while... So instead, let's take a sample of points.

Using sample size calculator, optimal size of sample is 385:

In [ ]:
sampled_rides = df.sample(385)

In [ ]:
map = folium.Map(
    location=[sampled_rides['start_lat'].mean(), sampled_rides['start_lng'].mean()],
    zoom_start=12,
    control_scale=True
)

In [ ]:
map

In [ ]:
start_ts = time.time()

for index, location_info in sampled_rides.iterrows():
    folium.Marker(
        [location_info['start_lat'], location_info['start_lng']],
        popup=location_info['end_station_name']
    ).add_to(map)


end_ts = time.time()
diff_ts = end_ts - start_ts
print(f'The fragment took {diff_ts:.3f} seconds, or {diff_ts/60:.3f} minutes.')

In [ ]:
map

Wow! Now we can see some examples of starting points. At a first glance, they seem to be closer to the coast line.

### Bordering stations

To confirm this, let's find the farthest points on all four sides.

In [ ]:
border_rides = pd.DataFrame()

In [ ]:
border_rides = pd.concat(
    [border_rides,
         df[df['start_lat'] == df['start_lat'].max()],
         df[df['start_lat'] == df['start_lat'].min()],
         df[df['start_lng'] == df['start_lng'].max()],
         df[df['start_lng'] == df['start_lng'].min()]
    ],
    axis=0,
    ignore_index=True
)

In [ ]:
border_rides.head(3)

In [ ]:
map_borders = folium.Map(
    location=[border_rides['start_lat'].mean(), border_rides['start_lng'].mean()],
    zoom_start=9,
    control_scale=True
)

start_ts = time.time()

for index, location_info in border_rides.iterrows():
    folium.Marker(
        [location_info['start_lat'], location_info['start_lng']],
        popup=location_info['end_station_name']
    ).add_to(map_borders)

end_ts = time.time()
diff_ts = end_ts - start_ts
print(f'The fragment took {diff_ts:.3f} seconds, or {diff_ts/60:.3f} minutes.')

In [ ]:
map_borders

All stations are located in the borders of Chicago area. Or not? Let's check with `Choropleth`.

In [ ]:
us_states = requests.get(
    "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
).json()

In [ ]:
folium.Choropleth(
    geo_data=us_states,
    fill_opacity=0.3,
    line_weight=2,
).add_to(map_borders);

In [ ]:
map_borders

It turns out the area is over *two* states – Illinois and Indiana. But the whole area can be classified as **Chicagoland**, the largest metropolitan area in the USA.

### Stations' popularity

Let's analyze density of some locations compared to other using Folium [heatmap](https://geopandas.org/en/stable/gallery/plotting_with_folium.html#Folium-Heatmaps):

In [ ]:
sampled_rides = df[['start_lat', 'start_lng']].sample(1_000_000)
sampled_rides.head()

In [ ]:
sampled_rides_coords = [[lat, lng] for lat, lng in zip(sampled_rides['start_lat'], sampled_rides['start_lng'])]

In [ ]:
map = folium.Map(
    location=[sampled_rides['start_lat'].mean(), sampled_rides['start_lng'].mean()],
    zoom_start=10,
    control_scale=True
)

In [ ]:
from folium import plugins # importing HeatMap

start_ts = time.time()

# adding coordinates to the heatmap
plugins.HeatMap(
    sampled_rides_coords
).add_to(map)

end_ts = time.time()
diff_ts = end_ts - start_ts
print(f'The fragment took {diff_ts:.3f} seconds, or {diff_ts/60:.3f} minutes.')

In [ ]:
map

Let's sample for `member` and `casual` riders and compare.

In [ ]:
start_ts = time.time()

# Taking a sample of casual riders
casual_rides = df[df['member_casual'] == 'casual'][['start_lat', 'start_lng']].sample(1_000_000)
casual_rides_coords = [[lat, lng] for lat, lng in zip(casual_rides['start_lat'], casual_rides['start_lng'])]

map_casual = folium.Map(
    location=[casual_rides['start_lat'].mean(), casual_rides['start_lng'].mean()],
    zoom_start=10,
    control_scale=True
)

# adding coordinates to the heatmap
plugins.HeatMap(
    casual_rides_coords
).add_to(map_casual)

end_ts = time.time()
diff_ts = end_ts - start_ts
print(f'The fragment took {diff_ts:.3f} seconds, or {diff_ts/60:.3f} minutes.')

In [ ]:
map_casual

In [ ]:
start_ts = time.time()

# Taking a sample of annual members
annual_rides = df[df['member_casual'] == 'member'][['start_lat', 'start_lng']].sample(1_000_000)
annual_rides_coords = [[lat, lng] for lat, lng in zip(annual_rides['start_lat'], annual_rides['start_lng'])]

map_annual = folium.Map(
    location=[annual_rides['start_lat'].mean(), annual_rides['start_lng'].mean()],
    zoom_start=10,
    control_scale=True
)

# adding coordinates to the heatmap
plugins.HeatMap(
    annual_rides_coords
).add_to(map_annual)

end_ts = time.time()
diff_ts = end_ts - start_ts
print(f'The fragment took {diff_ts:.3f} seconds, or {diff_ts/60:.3f} minutes.')

In [ ]:
map_annual

Very similar shapes (as expected), BUT it can be seen that annual members used farther stations (a blue circular figure on map). Moreover, the heatmap of annual members' station usage is a bit more extended on the ends, especially downwards.

### Visualizing routes

Inspired by this [post](https://towardsdatascience.com/visualizing-routes-on-interactive-maps-with-python-part-1-44f8d25d0761).

In [ ]:
data_5 = df.sample(10, random_state=44)
data_5

In [ ]:
map_routes = folium.Map(
    location=[data_5['start_lat'].mean(), data_5['start_lng'].mean()],
    zoom_start=13
)

for stop in data_5.itertuples():
    # Create markers for starting and ending stations
    start_marker = folium.Marker(
        location=(stop.start_lat, stop.start_lng),
        tooltip=stop.start_station_name
    )
    
    end_marker = folium.Marker(
        location=(stop.end_lat, stop.end_lng),
        tooltip=stop.end_station_name
    )
    
    # Make a route line connecting stations
    line = folium.PolyLine(
        locations=[(stop.start_lat, stop.start_lng), 
                   (stop.end_lat, stop.end_lng)],
        tooltip=f'{stop.start_station_name} to {stop.end_station_name}'
    )
    
    # Add all elements to the map
    start_marker.add_to(map_routes)
    end_marker.add_to(map_routes)
    line.add_to(map_routes)

In [ ]:
map_routes

Let's customize markers to differentiate routes:

In [ ]:
kw = {"prefix": "fa", "color": "green", "icon": "arrow-up"}

map_routes = folium.Map(
    location=[data_5['start_lat'].mean(), data_5['start_lng'].mean()],
    zoom_start=13
)

for stop in data_5.itertuples():
    # Create markers for starting and ending stations
    start_icon = folium.Icon(angle=180, **kw) # up
    start_marker = folium.Marker(
        location=(stop.start_lat, stop.start_lng),
        tooltip=stop.start_station_name,
        icon=start_icon
    )
    
    end_icon = folium.Icon(angle=0, **kw) # down
    end_marker = folium.Marker(
        location=(stop.end_lat, stop.end_lng),
        tooltip=stop.end_station_name,
        icon=end_icon
    )
    
    # Make a route line connecting stations
    line = folium.PolyLine(
        locations=[(stop.start_lat, stop.start_lng), 
                   (stop.end_lat, stop.end_lng)],
        tooltip=f'{stop.start_station_name} to {stop.end_station_name}'
    )
    
    # Add all elements to the map
    start_marker.add_to(map_routes)
    end_marker.add_to(map_routes)
    line.add_to(map_routes)

In [ ]:
map_routes

Better, but we cannot still see clearly which route belonged to which.

To address layered icons, let's add a little random margin to each and change colors:

In [ ]:
r_earth = 6371 # in km
pi = 3.1415

def jittered_location(lat, lng):
    "Return a location randomly shifted by +-100 meters."
    j_lat = lat + (np.random.uniform(low=0, high=0.1) / r_earth) * (180 / pi)
    j_lng = lng + (np.random.uniform(low=0, high=0.1) / r_earth) * (180 / pi) / np.cos(lat * pi/180)
    return [j_lat, j_lng]

In [ ]:
map_routes = folium.Map(
    location=[data_5['start_lat'].mean(), data_5['start_lng'].mean()],
    zoom_start=13
)

kw_start = {
    "prefix": "fa",
    "color": "green",
    "icon": "arrow-up"
}

kw_end = {
    "prefix": "fa",
    "color": "red",
    "icon": "arrow-up"
}

for stop in data_5.itertuples():
    # Create markers for starting and ending stations
    start_icon = folium.Icon(angle=0, **kw_start) # up
    start_marker = folium.Marker(
        location=jittered_location(stop.start_lat, stop.start_lng),
        tooltip=stop.start_station_name,
        icon=start_icon
    )
    
    end_icon = folium.Icon(angle=180, **kw_end) # down
    end_marker = folium.Marker(
        location=jittered_location(stop.end_lat, stop.end_lng),
        tooltip=stop.end_station_name,
        icon=end_icon
    )
    
    # Make a route line connecting stations
    line = folium.PolyLine(
        locations=[(stop.start_lat, stop.start_lng), 
                   (stop.end_lat, stop.end_lng)],
        tooltip=f'{stop.start_station_name} to {stop.end_station_name}'
    )
    
    # Add all elements to the map
    start_marker.add_to(map_routes)
    end_marker.add_to(map_routes)
    line.add_to(map_routes)

In [ ]:
map_routes

Perfect! Now we can see clearer where riders started and ended their routes.

In [ ]:
data_5 = df.sample(10, random_state=41)

map_routes = folium.Map(
    location=[data_5['start_lat'].mean(), data_5['start_lng'].mean()],
    zoom_start=13
)

kw_start = {
    "prefix": "fa",
    "color": "green",
    "icon": "arrow-up"
}

kw_end = {
    "prefix": "fa",
    "color": "red",
    "icon": "arrow-up"
}

for stop in data_5.itertuples():
    # Create markers for starting and ending stations
    start_icon = folium.Icon(angle=0, **kw_start) # up
    start_marker = folium.Marker(
        location=jittered_location(stop.start_lat, stop.start_lng),
        tooltip=stop.start_station_name,
        icon=start_icon
    )
    
    end_icon = folium.Icon(angle=180, **kw_end) # down
    end_marker = folium.Marker(
        location=jittered_location(stop.end_lat, stop.end_lng),
        tooltip=stop.end_station_name,
        icon=end_icon
    )
    
    # Make a route line connecting stations
    line = folium.PolyLine(
        locations=[(stop.start_lat, stop.start_lng), 
                   (stop.end_lat, stop.end_lng)],
        tooltip=f'{stop.start_station_name} to {stop.end_station_name}'
    )
    
    # Add all elements to the map
    start_marker.add_to(map_routes)
    end_marker.add_to(map_routes)
    line.add_to(map_routes)

map_routes

However, it is not efficient to visualize all observed routes.

Let's visualize 5 rides with the longest `distance`:

In [ ]:
data_5 = df.sort_values('distance', ascending=False).iloc[np.arange(5)]
data_5['distance']

In [ ]:
map_routes = folium.Map(
    location=[data_5['start_lat'].mean(), data_5['start_lng'].mean()],
    zoom_start=9
)

kw_start = {
    "prefix": "fa",
    "color": "green",
    "icon": "arrow-up"
}

kw_end = {
    "prefix": "fa",
    "color": "red",
    "icon": "arrow-up"
}

for stop in data_5.itertuples():
    # Create markers for starting and ending stations
    start_icon = folium.Icon(angle=0, **kw_start) # up
    start_marker = folium.Marker(
        location=jittered_location(stop.start_lat, stop.start_lng),
        tooltip=stop.start_station_name,
        icon=start_icon
    )
    
    end_icon = folium.Icon(angle=180, **kw_end) # down
    end_marker = folium.Marker(
        location=jittered_location(stop.end_lat, stop.end_lng),
        tooltip=stop.end_station_name,
        icon=end_icon
    )
    
    # Make a route line connecting stations
    line = folium.PolyLine(
        locations=[(stop.start_lat, stop.start_lng), 
                   (stop.end_lat, stop.end_lng)],
        tooltip=f'{stop.start_station_name} to {stop.end_station_name}'
    )
    
    # Add all elements to the map
    start_marker.add_to(map_routes)
    end_marker.add_to(map_routes)
    line.add_to(map_routes)

map_routes

Additionally, let's visualize 5 rides with the longest `duration`:

In [ ]:
data_5 = df.sort_values('duration', ascending=False).iloc[np.arange(5)]
data_5['duration']

In [ ]:
map_routes = folium.Map(
    location=[data_5['start_lat'].mean(), data_5['start_lng'].mean()],
    zoom_start=10
)

kw_start = {
    "prefix": "fa",
    "color": "green",
    "icon": "arrow-up"
}

kw_end = {
    "prefix": "fa",
    "color": "red",
    "icon": "arrow-up"
}

for stop in data_5.itertuples():
    # Create markers for starting and ending stations
    start_icon = folium.Icon(angle=0, **kw_start) # up
    start_marker = folium.Marker(
        location=jittered_location(stop.start_lat, stop.start_lng),
        tooltip=stop.start_station_name,
        icon=start_icon
    )
    
    end_icon = folium.Icon(angle=180, **kw_end) # down
    end_marker = folium.Marker(
        location=jittered_location(stop.end_lat, stop.end_lng),
        tooltip=stop.end_station_name,
        icon=end_icon
    )
    
    # Make a route line connecting stations
    line = folium.PolyLine(
        locations=[(stop.start_lat, stop.start_lng), 
                   (stop.end_lat, stop.end_lng)],
        tooltip=f'{stop.start_station_name} to {stop.end_station_name}'
    )
    
    # Add all elements to the map
    start_marker.add_to(map_routes)
    end_marker.add_to(map_routes)
    line.add_to(map_routes)

map_routes

### Stations' popularity, take 2

Let's make [`CircleMarker`](https://python-visualization.github.io/folium/latest/user_guide/vector_layers/circle_and_circle_marker.html) around each station with *counts* as an area parameter (given in meters).

In [ ]:
df.sample(3)

In [ ]:
df.groupby(['start_station_name', 'start_station_id', 'start_lat', 'start_lng']).count()

Note that rentals starting from the same station still have **different coordinates**.

To address this, let's take *only first* instance of coordinate and id.

In [ ]:
station_counts = (df[['start_station_name', 'start_station_id']]
    .groupby('start_station_name')
    .count()
    .rename({'start_station_id': 'count'}, axis='columns')
)
station_counts

In [ ]:
# Add id of each station
station_ids = []

for s in station_counts.index:
    sid = df[df['start_station_name'] == s].iloc[0]['start_station_id']
    station_ids.append(sid)
    
station_counts['id'] = station_ids

In [ ]:
# Add stations' coordinates
station_lats, station_lngs = [], []

for s in station_counts.index:
    lat = df[df['start_station_name'] == s].iloc[0]['start_lat']
    lng = df[df['start_station_name'] == s].iloc[0]['start_lng']
    station_lats.append(lat)
    station_lngs.append(lng)
    
station_counts['lat'] = station_lats
station_counts['lng'] = station_lngs

Took a long time, since it was initially searching through ALL rows for each unique value.

In [ ]:
np.any(station_counts.isna())

In [ ]:
# Make indices with stations' names a distinct column
stations = station_counts.rename_axis('name').reset_index()

In [ ]:
stations

Let's export this data to `./out` folder:

In [ ]:
stations.to_csv('out/stations.csv', index=False)

Now we are ready to start visualizing!

To set each circle's area proportional to the station popularity, we need to take into acccount **a large spread** in the counts, so let's take a log of it:

In [ ]:
np.min(stations['count']), np.mean(stations['count']), np.max(stations['count'])

In [ ]:
np.log(np.min(stations['count'])), np.log(np.mean(stations['count'])), np.log(np.max(stations['count']))

In [ ]:
# Create a new map
map_stations = folium.Map(
    location=[stations['lat'].mean(), stations['lng'].mean()],
    zoom_start=12
)

# Iterate through all stations
for station in stations.itertuples():
    radius_log = np.log(station.count)
    
    # Create a CircleMarker for the station
    station_marker = folium.CircleMarker(
        location=[station.lat, station.lng],
        tooltip=station.name,
        radius=radius_log,
        popup='{} rentals'.format(station.count),
        fill_color='green',
        color='green',
        fill_opacity=0.1,
        weight=1
    )
    
    # Add all elements to the map
    station_marker.add_to(map_stations)

In [ ]:
map_stations

Great! Let's now see which stations were used more by annual subscribers and casual riders.

In [ ]:
station_counts_by_member = (
    df[['start_station_name', 'start_station_id', 'member_casual']]
    .groupby(['start_station_name', 'member_casual'])
    .count()
    .rename({'start_station_id': 'count'}, axis='columns')
    .rename_axis(['name', 'type']).reset_index()
)
station_counts_by_member

In [ ]:
m_count, c_count = [], []

for s in stations.itertuples():
    s_rows = station_counts_by_member[station_counts_by_member['name'] == s.name]
    if s_rows.iloc[0]['type'] == 'member':
        member = s_rows.iloc[0]['count']
        casual = s.count - member
    else:
        casual = s_rows.iloc[0]['count']
        member = s.count - casual
    m_count.append(member)
    c_count.append(casual)
    
stations['member_count'] = m_count
stations['casual_count'] = c_count

In [ ]:
stations.sample(3)

Cool! Let's now indicate with colors which group of riders prevalently use this specific station:
green by annual members and red by casual riders.

In [ ]:
# Create a new map
map_stations = folium.Map(
    location=[stations['lat'].mean(), stations['lng'].mean()],
    zoom_start=12
)

# Iterate through all stations
for station in stations.itertuples():
    radius_log = np.log(station.count)
    if station.member_count >= station.casual_count:
        if station.member_count / station.count >= 0.75:
            color_ind = 'green'
        else:
            color_ind = 'yellow'
    else:
        if station.casual_count / station.count >= 0.75:
            color_ind = 'red'
        else:
            color_ind = 'yellow'
    
    # Create a CircleMarker for the station
    station_marker = folium.CircleMarker(
        location=[station.lat, station.lng],
        tooltip=station.name,
        radius=radius_log,
        popup='{} rentals'.format(station.count),
        fill_color=color_ind,
        color='yellow',
        fill_opacity=0.5,
        weight=1
    )
    
    # Add all elements to the map
    station_marker.add_to(map_stations)

In [ ]:
map_stations

Seems like casual riders tend to ride around the **downtown** area, while annual members outnumber in farther locations. This supports the conclusion we made from the heatmap earlier.

# Conclusion

Marketing strategy should focus on promoting accessibility to any point of the Chicago metropolitan area and speediness in order to make users buy an annual membership. These features seem to be characteristics of Cyclistic annual members, and could be the main reason for the purchase of membership.

---

Sardaana Eginova